In [1]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio wandb

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset, DatasetDict
medical_dataset = DatasetDict()
medical_dataset["train"] = load_dataset("yashtiwari/PaulMooney-Medical-ASR-Data", split="test")
medical_dataset["test"] = load_dataset("yashtiwari/PaulMooney-Medical-ASR-Data", split="train")

print(medical_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence', 'prompt', 'speaker_id', 'path'],
        num_rows: 5895
    })
    test: Dataset({
        features: ['id', 'sentence', 'prompt', 'speaker_id', 'path'],
        num_rows: 381
    })
})


In [4]:
medical_dataset = medical_dataset.remove_columns(["id", "prompt", "speaker_id"])
print(medical_dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'path'],
        num_rows: 5895
    })
    test: Dataset({
        features: ['sentence', 'path'],
        num_rows: 381
    })
})


In [5]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-medium")

In [6]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-medium")

In [7]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-medium")

In [8]:
from datasets import Audio
medical_dataset = medical_dataset.cast_column("path", Audio(sampling_rate=16000))

In [9]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["path"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [10]:
medical_dataset = medical_dataset.map(prepare_dataset, remove_columns=medical_dataset.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/5895 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/381 [00:00<?, ? examples/s]

In [11]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

In [12]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [14]:
import evaluate

metric = evaluate.load("wer")

In [15]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [16]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-medium-medical",  # change to a repo name of your choice
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=50,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=100,
    logging_steps=50,
    report_to=["wandb"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    run_name="whisper-medium-medical"
)

/root/miniconda3/envs/WhisperFinetuning/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=medical_dataset["train"],
    eval_dataset=medical_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/root/miniconda3/envs/WhisperFinetuning/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [18]:
processor.save_pretrained(training_args.output_dir)

[]

In [19]:
trainer.train()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
/root/miniconda3/envs/WhisperFinetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss,Wer
100,0.485900,0.194520,15.592566
200,0.103700,0.084928,12.575428
300,0.055800,0.063316,17.378711
400,0.024400,0.063065,13.758146
500,0.012300,0.057704,16.027034


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/root/miniconda3/envs/WhisperFinetuning/lib/python3.10/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253,

TrainOutput(global_step=500, training_loss=0.43465084552764893, metrics={'train_runtime': 3014.6215, 'train_samples_per_second': 5.307, 'train_steps_per_second': 0.166, 'total_flos': 1.6278648274944e+19, 'train_loss': 0.43465084552764893, 'epoch': 2.7027027027027026})

In [20]:
kwargs = {
    "dataset_tags": "yashtiwari/PaulMooney-Medical-ASR-Data",
    "dataset": "Medical ASR",  
    "model_name": "Whisper Medium Medical",  
    "finetuned_from": "openai/whisper-medium",
    "tasks": "automatic-speech-recognition",
}

In [21]:
trainer.push_to_hub(**kwargs)

CommitInfo(commit_url='https://huggingface.co/davidwei111/whisper-medium-medical/commit/8ebd6305c5d862a276b5f1878146c0609e77b514', commit_message='End of training', commit_description='', oid='8ebd6305c5d862a276b5f1878146c0609e77b514', pr_url=None, repo_url=RepoUrl('https://huggingface.co/davidwei111/whisper-medium-medical', endpoint='https://huggingface.co', repo_type='model', repo_id='davidwei111/whisper-medium-medical'), pr_revision=None, pr_num=None)

In [ ]:
!sudo apt update  
!sudo apt install ffmpeg

In [25]:
from transformers import pipeline  
import gradio as gr  
  
# 加载模型  
pipe = pipeline(model="davidwei111/whisper-medium-medical")  # change to "your-username/the-name-you-picked"  
  
def transcribe(audio):  
    # 检查音频文件路径是否有效  
    if audio is None:  
        return "Error: No audio file provided."  
      
    # 尝试读取音频文件并进行转录  
    try:  
        text = pipe(audio)["text"]  
        return text  
    except Exception as e:  
        return f"Error: {str(e)}"  
  
# 配置Gradio接口  
iface = gr.Interface(  
    fn=transcribe,  
    inputs=gr.Audio(type="filepath"),  
    outputs="text",  
    title="Whisper Medical ASR",  
    description="Demo of OpenAI Whisper on medical transcription dataset",  
)  
  
# 启动接口  
iface.launch(share=True)  


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://28adece02db85e641d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/root/miniconda3/envs/WhisperFinetuning/lib/python3.10/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
Traceback (most recent call last):
  File "/root/miniconda3/envs/WhisperFinetuning/lib/python3.10/site-packages/gradio/processing_utils.py", line 536, in audio_from_file
    audio = AudioSegment.from_file(filename)
  File "/root/miniconda3/envs/WhisperFinetuning/lib/python3.10/site-packages/pydub/audio_segment.py", line 728, in from_file
    info = mediainfo_json(orig_file, read_ahead_limit=read_ahead_limit)
  File "/root/miniconda3/envs/WhisperFinetuning/lib/python3.10/site-packages/pydub/utils.py", line 274, in mediainfo_json
    res = Popen(command, stdin=stdin_parameter, stdout=PIPE, stderr=PIPE)
  File "/root/miniconda3/envs/WhisperFinetuning/lib/python3.10/subprocess.py", line 971, in __init__
    self._